## Module 4 - Dans ce notebook jupyter, les productivités de la terre et de l'eau sont calculées
* Etape 4a - Réglage
* Etape 4b - Calculer la productivité de la terre: i) de la biomasse et ii) le rendement de la culture  
* Etape 4c - Calculer la : i) productivité en biomasse de l'eau et ii) la productivité culturale de l'eau
**=====================================================================================================================**

![title](img/Fig4_1.png)

**=====================================================================================================================**

## Etape 4a - Réglage

## i) Importer les packages/librairies

In [1]:
import os
import sys
import glob

import numpy as np
from matplotlib import pyplot as plt

# changer le répertoire à là ou les modules sont enregistrés
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## Etape 4b - Calculer la productivité de la terre
* i) Biomasse
* ii) Rendement de la culture

## i) Calculer la biomasse
$Biomass   = AOT*f_c*\frac{NPP*22.222}{1-mc}$
<br/>Avec AOT est le ratio de la production de biomasse au dessus du sol sur la production totale (-), fc est le facteur de correction de l'efficacité d'utilisation de la lumière[-]. NPP est la production primaire nette saisonnière en gC/m²/saison, mc est la teneur en eau de la biomasse fraiche [-] et HI l'indice de recolte

## * Importer les données raster de la NPP saisonnière

In [2]:
dir_proj = os.path.split(os.getcwd())[0]  # os.getcwd() est pour le repertoire actuel
dir_data = "Data"

input_folderNPP = os.path.join(dir_proj, dir_data, "2L2_NPP_season") 
input_fhsNPP    = glob.glob(input_folderNPP + '\*.tif')
input_fhsNPP

## ** Parametres de la culture
![title](img/Fig4_2.png)

In [3]:
# Mettre à jour les paramètres culturaux spécifique à la culture
MC = 0.7  # teneur en eau, matière sèche sur biomasse fraiche
fc = 1.6  # Facteur de correction de l'efficacité d'utilisation de la lumière
AOT= 0.8  # le ratio de la production de biomasse au dessus du sol sur la production totale(AOT)

## *** Dossier de sortie pour la couche de la biomasse: Créer un ou connecter l'existant

In [4]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folderBM = os.path.join(dir_proj, dir_data, "4L2_Biomass") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folderBM):
    os.makedirs(output_folderBM) 
output_folderBM

## **** Calculer la biomasse au dessus du sol (AGBM)

In [5]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsNPP[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster


for in_fhNPP in input_fhsNPP:
    NPP  = gis.OpenAsArray(in_fhNPP, nan_values=True)  
    AGBM = (AOT * fc * (NPP * 22.222 / (1 - MC))) / 1000  # Biomasse au dessus du sol, 1000 pour convertir de kg en tonne  
    
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhNPP).split('.')[0].replace('NPP', '').replace('_', ' ')
                             
    # enregistrer dans le dossier de sortie
    output_folder = output_folderBM                 
    basename      = os.path.basename(in_fhNPP).replace('NPP', 'AGBM')                   
    output_fn     = os.path.join(output_folder,basename)                   
    gis.CreateGeoTiff(output_fn, AGBM, driver, NDV, xsize, ysize, GeoT, Projection) 
                             
    
    # Tracer la carte du raster
    seasonal = AGBM
    
    # calculer la moyenne, et l'écart type
    print ('the mean & SD for ',date, '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='Above ground biomass [ton/ha/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Above ground biomass [ton/ha/season] ' + date)
    plt.clim(30,80)
    plt.show ()
    ;                             

## ii) Calculer le rendement de la culture
* $Rendement de la culture = HI*B$
* Avec HI l'indice de recolte, et B (ou AGBM) la biomasse au dessus du sol

## * Importer les données raster d'entrée

In [6]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

input_folder  = os.path.join(dir_proj, dir_data, "4L2_Biomass") 
input_fhsAGBM = glob.glob(input_folder + '\*.tif')
input_fhsAGBM

## ** Mettre à jour l'indice de recolte

In [7]:
HI = 1     # Indice de recolte de la culture

## ***  Dossier de sortie pour la couche du rendement de la culture: Créer un ou connecter l'existant

In [8]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folderY = os.path.join(dir_proj, dir_data, "4L2_Yield") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folderY):
    os.makedirs(output_folderY) 
output_folderY

## **** Calculer le rendement de la culture

In [9]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsAGBM[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for in_fhAGBM in input_fhsAGBM:
    AGBM      = gis.OpenAsArray(in_fhAGBM, nan_values=True)  
    CropYield = HI*AGBM  
        
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhAGBM).split('.')[0].replace('AGBM', '').replace('_', ' ')
                             
    # enregistrer dans le dossier de sortie
    output_folder = output_folderY                 
    basename      = os.path.basename(in_fhAGBM).replace('AGBM', 'Yield')                   
    output_fn     = os.path.join(output_folder,basename)                   
    gis.CreateGeoTiff(output_fn, CropYield, driver, NDV, xsize, ysize, GeoT, Projection) 
                             
    # Tracer la carte du raster
    seasonal = CropYield
       
    # calculer la moyenne, et l'écart type
    print ('the mean & SD for ',date, '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='Crop yield [ton/ha/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Crop yield [ton/ha/season] ' + date)
    plt.clim()
    plt.show ()
    ;                             

## Etape 4c - Calculer la productivité de l'eau (PE)

## i) Calculer la productivité en biomasse de l'eau 
$PE Biomasse= \frac{Biomasse}{ET_a}$

## * Importer les données raster d'entrée

In [10]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

input_folder1  = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
input_fhsET = glob.glob(input_folder1 + '\*.tif')
input_folder2  = os.path.join(dir_proj, dir_data, "4L2_Biomass") 
input_fhsAGBM = glob.glob(input_folder2 + '\*.tif')
input_fhsET[0], input_fhsAGBM[0]

## ** Dossier de sortie pour la couche de la PEb: Créer un ou connecter l'existant

In [11]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folderWPb = os.path.join(dir_proj, dir_data, "4L2_WPb") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folderWPb):
    os.makedirs(output_folderWPb) 
output_folderWPb

## **** Calculer la productivité en biomasse de l'eau (PEb)

In [12]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

## Calculer la PE
for in_fhET, in_fhAGBM in zip(input_fhsET, input_fhsAGBM):
    AETI = gis.OpenAsArray(in_fhET, nan_values=True) 
    AGBM = gis.OpenAsArray(in_fhAGBM, nan_values=True)
    WPb  = AGBM/AETI*100
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhET).split('.')[0].replace('AETI', '').replace('_', ' ')
    
    # enregistrer dans le dossier de sortie
    output_folder = output_folderWPb
    basename      = os.path.basename(in_fhET).replace('AETI', 'WPb')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn, WPb, driver, NDV, xsize, ysize, GeoT, Projection) 
        
    # Tracer la carte du raster
    seasonal = WPb
    
    # calculer la moyenne, et l'écart type
    print ('the mean & SD for ',date, '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='WPb [kg/m3]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Biomass water productivity [kg/m3] ' + date)
    plt.clim(2,6)
    plt.show ()  
    ;

## ii) Calculer la productivité culturale de l'eau 
$PE culture = \frac{Rendement de la culture}{ET_a}$

## * Importer les données raster d'entrée

In [13]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

input_folder1  = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
input_fhsET    = glob.glob(input_folder1 + '\*.tif')
input_folder2  = os.path.join(dir_proj, dir_data, "4L2_Yield") 
input_fhsY     = glob.glob(input_folder2 + '\*.tif')
input_fhsET[0], input_fhsY[0]

## ** Dossier de sortie pour la couche de la PEr: Créer un ou connecter l'existant

In [14]:
dir_proj = os.path.split(os.getcwd())[0]   
dir_data = "Data"

output_folderWPy = os.path.join(dir_proj, dir_data, "4L2_WPy") 

# si le repertoire et le dossier n'existent pas, en créer
if not os.path.exists(output_folderWPy):
    os.makedirs(output_folderWPy) 
output_folderWPy

## **** Calculer la productivité culturale de l'eau (WPr)

In [15]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = input_fhsET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

## Calculer la PE
for in_fhET, in_fhY in zip(input_fhsET, input_fhsY):
    AETI  = gis.OpenAsArray(in_fhET, nan_values=True) 
    Yield = gis.OpenAsArray(in_fhY, nan_values=True)
    WPy   = Yield/AETI*100

    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(in_fhET).split('.')[0].replace('AETI', '').replace('_', ' ')

    # enregistrer dans le dossier de sortie
    output_folder = output_folderWPy
    basename      = os.path.basename(in_fhET).replace('AETI', 'WPy')
    output_fn     = os.path.join(output_folder, basename)
    gis.CreateGeoTiff(output_fn, WPy, driver, NDV, xsize, ysize, GeoT, Projection) 

    # Tracer la carte du raster
    seasonal = WPy
    
    # calculer la moyenne, et l'écart type
    print ('the mean & SD for ',date, '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='Crop water productivity [kg/m3]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Crop water productivity [kg/m3] ' + date)
    plt.clim(2,6)
    plt.show ()   
    ;